## Copy PDFs

In [1]:
from os import environ, path
from aa_scrape_PDF import copy_pdfs

environ["_RAG_PDF_SOURCE"] = "/media/james/FILEPILE/$_Robust_Planning/Literature/References/storage"
environ["_RAG_PDF_DESTIN"] = "data/input/pdf"
environ["_RAG_VERBOSE"]    = ""
environ["_RAG_DOC_LIMIT"]  = "250"
environ["_RAG_DOC_DBASE"] = "lit_pdf"
environ["_RAG_VEC_DBASE"] = "lit_vec"
copy_pdfs()

.....


## Load PDFs by page chunks

### https://python.langchain.com/docs/how_to/document_loader_pdf/

* `python3.10 -m pip install pypdf langchain-unstructured "unstructured[pdf]" --user`
* `apt install tesseract-ocr`

In [2]:
import os
from collections import deque
from langchain_community.document_loaders import PyPDFLoader

pdfs_drct = environ["_RAG_PDF_DESTIN"]
fNames    = [item for item in os.listdir( pdfs_drct ) if (str( item ).split('.')[-1].lower() == 'pdf')]
print( f"Copied {len(fNames)} files!" )
pages = deque() # Fast append

for i, fNam in enumerate( fNames ):
    file_path = path.join( pdfs_drct, fNam )
    loader    = PyPDFLoader( file_path )
    async for page in loader.alazy_load():
        pages.append( page )
    print( f"{i+1}:{len(pages)}", end = ', ', flush = True )
print()
pages = list( pages )

Copied 251 files!
1:9, 2:29, 3:44, 4:56, 5:85, 6:93, 7:104, 8:109, 9:124, 10:130, 11:142, 

Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced encoding /90msp-RKSJ-H not implemented yet
Advanced enc

12:154, 13:161, 14:184, 15:198, 16:272, 17:285, 18:301, 19:326, 20:369, 21:377, 22:378, 23:388, 24:408, 25:419, 26:424, 27:441, 28:455, 29:463, 30:474, 31:483, 32:488, 33:502, 34:527, 35:539, 36:548, 37:562, 

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 22 65536 (offset 0)
Ignoring wrong pointing object 26 65536 (offset 0)
Ignoring wrong pointing object 46 65536 (offset 0)
Ignoring wrong pointing object 52 65536 (offset 0)
Ignoring wrong pointing object 57 65536 (offset 0)
Ignoring wrong pointing object 61 65536 (offset 0)
Ignoring wrong pointing object 73 65536 (offset 0)
Ignoring wrong pointing object 78 65536 (offset 0)
Ignoring wrong pointing object 83 65536 (offset 0)
Ignoring wrong pointing object 88 65536 (offset 0)
Ignoring wrong pointing object 97 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 107 65536 (offset 0)
Ignoring wrong pointing object 112 65536 (offset 0)
Ignoring wrong pointing object 121 65536 (offset 0)
Ignoring wrong pointing object 126 65536 (offset 0)
Ignoring wrong pointing object 135 65536 (offset 0)
Ignoring wrong pointing object 140 65536 (offset 0)
Ignoring wrong pointing o

38:690, 39:706, 40:738, 41:746, 42:753, 43:821, 44:830, 45:865, 

...

53:994, 54:1014, 55:1022, 56:1036, 57:1094, 58:1122, 59:1125, 60:1141, 61:1154, 62:1164, 63:1189, 64:1202, 65:1224, 66:1243, 67:1269, 68:1279, 69:1284, 70:1314, 71:1322, 72:1695, 73:1720, 74:1733, 75:1741, 76:1747, 77:1751, 78:1766, 79:1771, 80:1793, 81:1803, 82:1809, 83:1841, 84:1849, 85:1863, 86:1885, 87:1897, 88:1914, 89:1927, 90:1935, 91:1972, 92:1991, 93:2002, 94:2022, 95:2030, 96:2037, 97:2043, 98:2058, 99:2071, 100:2077, 101:2089, 102:2098, 103:2124, 104:2130, 105:2139, 106:2170, 107:2177, 108:2183, 109:2197, 110:2619, 111:2627, 112:2639, 113:2655, 114:2662, 115:2677, 116:2689, 117:2716, 118:2724, 119:2737, 120:2760, 121:2776, 122:2781, 123:2788, 124:2798, 125:2805, 126:2813, 127:2845, 128:2854, 129:2863, 130:2879, 131:2904, 132:3452, 133:3466, 134:3473, 135:3495, 136:3525, 137:3546, 138:3566, 139:3596, 140:3616, 141:3622, 142:3628, 143:3647, 144:3665, 145:3674, 146:3683, 147:3687, 148:3697, 149:3705, 150:3713, 151:3723, 152:3737, 153:3748, 154:3757, 155:3764, 156:3771, 157:3802

Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 110 0 (offset 0)
Ignoring wrong pointing object 121 0 (offset 0)
Ignoring wrong pointing object 128 0 (offset 0)
Ignoring wrong pointing object 153 0 (offset 0)
Ignoring wrong pointing object 159 0 (offset 0)
Ignoring wrong pointing object 212 0 (offset 0)
Ignoring wrong pointing object 234 0 (offset 0)
Ignoring wrong pointing object 269 0 (offset 0)
Ignoring wrong pointing object 337 0 (offset 0)
Ignoring wrong pointing object 339 0 (offset 0)
Ignoring wrong pointing object 355 0 (offset 0)
Ignoring wrong pointing object 361 0 (offset 0)
Ignoring wrong pointing object 365 0 (offset 0)
Ignoring wrong pointing object 367 0 (offset 0)

194:5107, 195:5130, 196:5148, 197:5154, 198:5170, 199:5195, 200:5208, 201:5217, 202:5228, 203:5240, 204:5248, 205:5267, 206:5293, 207:5315, 

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 122 0 (offset 0)
Ignoring wrong pointing object 127 0 (offset 0)
Ignoring wrong pointing object 132 0 (offset 0)
Ignoring wrong pointing object 134 0 (offset 0)
Ignoring wrong pointing object 143 0 (offset 0)
Ignoring wrong pointing object 148 0 (offset 0)
Ignoring wrong pointing object 153 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 223 0 (offset 0)
Ignoring wrong pointing object 225 0 (offset 0)
Ignoring wrong pointing object 227 0 (offset 0)
Ignoring wrong pointing object 229 0 (offset 0)
Ignoring wrong pointing object 231 0 (offset 0)
Ignoring wrong pointing object 233 0 (offset 0)
Ignoring wrong pointing object 235 0 (offset 0)
Ignoring wrong pointing object 237 0 (offset 0)
Ignoring wrong pointing object 239 0 (offset

208:5356, 209:5380, 210:5403, 211:5412, 212:5424, 213:5438, 214:5446, 215:5452, 216:5463, 217:5832, 218:5839, 219:5853, 220:5860, 221:5878, 222:5888, 223:5895, 224:5901, 225:5907, 226:5923, 227:5942, 228:5958, 229:5968, 230:5980, 231:5988, 232:6000, 233:6008, 234:6024, 235:6046, 236:6069, 237:6111, 238:6121, 239:6145, 240:6156, 241:6222, 242:6230, 243:6238, 244:6249, 245:6304, 246:6336, 247:6355, 248:6364, 249:6385, 250:6406, 251:6418, 


In [3]:
# print(f"{pages[0].metadata}\n")
# print(pages[0].page_content)

## Load the text embedding model

In [4]:
def pull_ollama_model( modelStr ):
    """ Pull a named model from Ollama and store it wherever """
    print( f"About to save '{modelStr}'.\nThis will spew a lot of text on the first run..." )
    os.system( f"ollama pull {modelStr}" )

In [5]:

import sys, os, time
now = time.time


# from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
# from langchain_community import embeddings

pull_ollama_model( "nomic-embed-text" )

local_embeddings = OllamaEmbeddings( model = "nomic-embed-text" )

About to save 'nomic-embed-text'.
This will spew a lot of text on the first run...


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


# Setup document database (of pages)

In [12]:
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules.pop( 'pysqlite3' )
import chromadb
from langchain_chroma import Chroma
from uuid import uuid4 as gen_id

docIDs = [str( gen_id() ) for _ in range( len(pages) )]
dcmnts = [str( pg.page_content ) for pg in pages]

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection( environ["_RAG_DOC_DBASE"] )
collection.add(
    ids       = docIDs, 
    documents = dcmnts
)



KeyboardInterrupt: 

# Create vector store

In [11]:
bgn = now()
vector_store_from_client = Chroma(
    client             = persistent_client,
    collection_name    = environ["_RAG_DOC_DBASE"],
    embedding_function = local_embeddings,
)
print( f"Built vector store in {now()-bgn} seconds!" )

Built vector store in 0.0033881664276123047 seconds!


In [10]:
# vectorstore      = Chroma.from_documents( documents = pages, embedding = local_embeddings )
# print( f"Built vector store in {now()-bgn} seconds!" )

Exception occurred invoking consumer for subscription 94c9cf52038f4a5194039d5772c7c84eto topic persistent://default/default/a72b354b-23c5-415d-b4dc-c6fdd85a6258 'utf-8' codec can't encode character '\udc38' in position 1416: surrogates not allowed


NameError: name 'bgn' is not defined

### 2024-10-18: Langchain's API Key fails

In [ ]:
# import getpass
# import os

# _UNSTRUCT_KEY_LOC = "secrets/Langchain_Unstructured.txt"

# with open( _UNSTRUCT_KEY_LOC, 'r' ) as f:
#     os.environ["UNSTRUCTURED_API_KEY"] = str( f.read() ).strip()
    

In [ ]:
# from langchain_unstructured import UnstructuredLoader

# loader = UnstructuredLoader(
#     file_path = file_path,
#     strategy  = "hi_res",
#     partition_via_api = True,
#     coordinates = True,
# )
# docs = []
# for doc in loader.lazy_load():
#     docs.append( doc )

### https://python.langchain.com/docs/how_to/document_loader_pdf/#local-parsing

In [ ]:
from langchain_unstructured import UnstructuredLoader

loader_local = UnstructuredLoader(
    file_path = file_path,
    strategy  = "fast", #"hi_res",
)
docs_local = []
for doc in loader_local.lazy_load():
    docs_local.append( doc )

In [ ]:
len( docs_local )

In [ ]:
from langchain_ollama import ChatOllama

pull_ollama_model( "llava" )

llm = ChatOllama(
    model="llava",
)

In [ ]:
import base64
import io

import fitz
from PIL import Image


def pdf_page_to_base64(pdf_path: str, page_number: int):
    pdf_document = fitz.open(pdf_path)
    page = pdf_document.load_page(page_number - 1)  # input is one-indexed
    pix = page.get_pixmap()
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    buffer = io.BytesIO()
    img.save(buffer, format="PNG")

    return base64.b64encode(buffer.getvalue()).decode("utf-8")

In [ ]:
from IPython.display import Image as IPImage
from IPython.display import display

base64_image = pdf_page_to_base64(file_path, 11)
display( IPImage( data = base64.b64decode( base64_image ) ) )

In [ ]:
from langchain_core.messages import HumanMessage

query = "What can be said about the data composition?"

message = HumanMessage(
    content=[
        {"type": "text", "text": query},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
        },
    ],
)
bgn = now()
response = llm.invoke( [message] )
print( f"LLM query took {now()-bgn} seconds to process!" )
print( response.content )